In [6]:
import pandas as pd
import numpy as np
from agcounts.extract import get_counts

import pa_proc

## Step 0: Convert ExpoApp data to Actigraph-like data 

In [10]:
# Specify the path of your file
import pa_proc.expoapp_to_acc


path_expoapp_raw = "C:/Users/jbuekers/Desktop/Sab_339-Mob1-1/"

# Create dataframe
data_act = pa_proc.expoapp_to_acc.expoapp_to_actigraph(path_expoapp_raw + "acc.csv", 1, "Europe/Madrid")

# Save dataframe to csv
data_act.to_csv(path_expoapp_raw + "/acc_prep2.csv",index=False, header=False)

NameError: name 'pd' is not defined

## Step 1: Convert Actigraph-like data to counts and ENMO

In [3]:
# Specify path of where data is saved
path = "C:/Users/jbuekers/Desktop/Sab_339-Mob1-1/"

[enmo_10s, counts_enmo] = acc_to_counts_enmo(path + "acc_prep2.csv", 'expoapp', 30, 1)

## Step 2: calculate physical activity variables

In [4]:

# Identify different days in data
days_measured = counts_enmo.index.map(pd.Timestamp.date).unique()
 
# Create results dataframe
results = pd.DataFrame(index=days_measured)
 
# Calculate time in different PA domains (cut-offs used from document Gabriela)
for day_loop in days_measured:
	
	day_data = counts_enmo[(counts_enmo.index.date == day_loop)]
	
	results.loc[day_loop, 'weartime_choi'] = np.sum(day_data.wear_choi == 1)
	results.loc[day_loop, 'weartime_vanhees'] = np.sum(day_data.wear_vanhees == 1)
	
	day_data_wear_choi = day_data[day_data.wear_choi == 1]
	day_data_wear_vanhees = day_data[day_data.wear_vanhees == 1]
	
	results.loc[day_loop, 'counts_lpa'] = np.sum((day_data_wear_choi.counts_vm > 200) & (day_data_wear_choi.counts_vm <= 2690))
	results.loc[day_loop, 'counts_mvpa'] = np.sum(day_data_wear_choi.counts_vm > 2690)
	results.loc[day_loop, 'counts_vpa'] = np.sum(day_data_wear_choi.counts_vm > 6166)
 
 
	results.loc[day_loop, 'enmo_lpa'] = np.sum((day_data_wear_vanhees.enmo > 47.4) & (day_data_wear_vanhees.enmo < 69.1))
	results.loc[day_loop, 'enmo_mvpa'] = np.sum(day_data_wear_vanhees.enmo >= 69.1)
	results.loc[day_loop, 'enmo_vpa'] = np.sum(day_data_wear_vanhees.enmo >= 258.7)

